# Create info that will be displayed in the map
we re-use some code from the notebook 'Descriptive Analysis' here

In [1]:
import json

In [2]:
import pg8000

In [3]:
import pandas as pd

In [4]:
import numpy as np

## 1 Get the data out of the database

In [70]:
conn = pg8000.connect(database="parl", user="webapp", password="webapp")

* affairs

In [49]:
query = "SELECT affairs.affair_id, affairs.affair_type, affairs.council, affairs.legislative_period, affairs.language, affairs.affair_title, affairs.affair_text, affairs.answer, "
query += "councillors.councillor_id, councillors.firstname, councillors.lastname, councillors.canton, councillors.party "
query += "FROM affairs JOIN councillors ON affairs.councillor_id = councillors.councillor_id "
query += "WHERE affairs.affair_type = '14' AND affairs.legislative_period = '50' "
query += "ORDER BY affairs.affair_id;"
query

"SELECT affairs.affair_id, affairs.affair_type, affairs.council, affairs.legislative_period, affairs.language, affairs.affair_title, affairs.affair_text, affairs.answer, councillors.councillor_id, councillors.firstname, councillors.lastname, councillors.canton, councillors.party FROM affairs JOIN councillors ON affairs.councillor_id = councillors.councillor_id WHERE affairs.affair_type = '14' AND affairs.legislative_period = '50' ORDER BY affairs.affair_id;"

In [50]:
df = pd.read_sql(query, conn)

In [51]:
df.head(1)

,b'affair_id',b'affair_type',b'council',b'legislative_period',b'language',b'affair_title',b'affair_text',b'answer',b'councillor_id',b'firstname',b'lastname',b'canton',b'party'
0,20155540,14,NR,50,de,Prinzip der Wahrung der Interessen der Schweiz...,<p>Die Wahrung der Interessen der Schweiz in d...,,4025,Roland Rino,Büchel,SG,SVP


In [52]:
df_colnames = {
    b'affair_id': 'affair_id',
    b'affair_type': 'affair_type',
    b'council': 'council',
    b'legislative_period': 'legislative_period',
    b'language': 'language',
    b'affair_title': 'affair_title',
    b'affair_text': 'affair_text',
    b'answer': 'answer',
    b'councillor_id': 'councillor_id',
    b'firstname': 'firstname',
    b'lastname': 'lastname',
    b'party': 'party',
    b'canton': 'canton'
}

In [53]:
df = df.rename(columns=df_colnames)

In [54]:
df.head(1)

,affair_id,affair_type,council,legislative_period,language,affair_title,affair_text,answer,councillor_id,firstname,lastname,canton,party
0,20155540,14,NR,50,de,Prinzip der Wahrung der Interessen der Schweiz...,<p>Die Wahrung der Interessen der Schweiz in d...,,4025,Roland Rino,Büchel,SG,SVP


* the councillors

In [71]:
query2 = "SELECT councillors.councillor_id, councillors.firstname, councillors.lastname, councillors.canton, councillors.party "
query2 += "FROM councillors;"
query2

'SELECT councillors.councillor_id, councillors.firstname, councillors.lastname, councillors.canton, councillors.party FROM councillors;'

In [72]:
df_c = pd.read_sql(query2, conn)

In [73]:
df_c.head(1)

,b'councillor_id',b'firstname',b'lastname',b'canton',b'party'
0,4154,Jean-Luc,Addor,VS,SVP


In [74]:
df_c_colnames = {
    b'councillor_id': 'councillor_id',
    b'firstname': 'firstname',
    b'lastname': 'lastname',
    b'party': 'party',
    b'canton': 'canton'
}

In [75]:
df_c = df_c.rename(columns=df_c_colnames)

In [76]:
df_c.head(1)

,councillor_id,firstname,lastname,canton,party
0,4154,Jean-Luc,Addor,VS,SVP


In [77]:
conn.close()

## 2 Prepare json-like structures

* Define the canton_factor - how many questions per councillor in each canton. This will be used as the basis for the coloring of the cantons in the map

In [62]:
cantons_hitlist = df['canton'].value_counts()
cantons_numcounc = df_c['canton'].value_counts()
cantons_factor = cantons_hitlist / cantons_numcounc

In [63]:
#some cleaning up is necessary
cantons_factor.fillna(0, inplace=True)
cantons_factor = round(cantons_factor, 1).astype(str)

In [64]:
cantons_factor

AG     5.6
AI     1.0
AR    11.0
BE     4.5
BL    11.1
BS     4.0
FR     2.4
GE     8.2
GL     0.0
GR     2.4
JU     4.5
LU     3.5
NE     3.0
NW    15.0
OW    12.0
SG     5.2
SH     7.5
SO     6.5
SZ     2.5
TG     2.0
TI     7.1
UR     7.0
VD     4.7
VS     7.1
ZG     7.7
ZH     4.1
Name: canton, dtype: object

* create a dictionary for each canton that has a dictionary for each councillor, containing id, name, party, number of questions

In [78]:
#To start, we construct the full name
df_c['fullname'] = df_c['firstname'] + " " + df_c['lastname']

In [79]:
c_dicts = df_c[['councillor_id', 'fullname', 'party', 'canton']]

In [80]:
c_dicts.head()

,councillor_id,fullname,party,canton
0,4154,Jean-Luc Addor,SVP,VS
1,3867,Andreas Aebi,SVP,BE
2,4049,Matthias Aebischer,SP,BE
3,4053,Thomas Aeschi,SVP,ZG
4,1131,Evi Allemann,SP,BE


In [81]:
c_values = df['councillor_id'].value_counts().to_frame().rename(columns={'councillor_id': 'num_questions'})
c_values.head()

,num_questions
1120,31
4154,29
487,27
4090,23
173,23


In [82]:
c_dicts = c_dicts.join(c_values, on='councillor_id')
c_dicts.fillna(0, inplace=True)

In [83]:
c_dicts.head()

,councillor_id,fullname,party,canton,num_questions
0,4154,Jean-Luc Addor,SVP,VS,29
1,3867,Andreas Aebi,SVP,BE,8
2,4049,Matthias Aebischer,SP,BE,2
3,4053,Thomas Aeschi,SVP,ZG,17
4,1131,Evi Allemann,SP,BE,1


# 3 Load the GeoJSON and add the prepared structures to it

* Load the GeoJSON of the Swiss cantons from a file we found on the internet

In [84]:
with open('schweizer_kantone.geojson') as json_data:
    cantons_json = json.load(json_data)

In [85]:
cantons_list = cantons_json['features']

In [86]:
cantons_list[0].keys()

dict_keys(['type', 'geometry', 'properties'])

In [87]:
cantons_list[0]['properties']

{'area_ha': 83851, 'cartodb_id': 20, 'kanton': 'JU'}

* Make some modifications in the 'properties' key of each feature:
- Add canton names to the abbreviations
- drop unnecessary other stuff

In [88]:
cant_ids = {
    'JU': 'Jura',
    'NE': 'Neuenburg',
    'BE': 'Bern',
    'GR': 'Graubünden',
    'VD': 'Waadt',
    'BS': 'Basel-Stadt',
    'ZG': 'Zug',
    'NW': 'Nidwalden',
    'AI': 'Appenzell Innerrhoden',
    'LU': 'Luzern',
    'SZ': 'Schwyz',
    'UR': 'Uri',
    'OW': 'Obwalden',
    'GL': 'Glarus',
    'BL': 'Basel-Landschaft',
    'SO': 'Solothurn',
    'SH': 'Schaffhausen',
    'AR': 'Appenzell Ausserrhoden',
    'SG': 'St. Gallen',
    'AG': 'Aargau',
    'GE': 'Genf',
    'TG': 'Thurgau',
    'ZH': 'Zürich',
    'FR': 'Freiburg',
    'VS': 'Wallis',
    'TI': 'Tessin'
}

In [89]:
for canton in cantons_list:
    canton['properties'].pop('cartodb_id')
    canton['properties'].pop('area_ha')
    canton['properties']['kanton_long'] = cant_ids[canton['properties']['kanton']]

*then, we insert the ratio of questions/councillors for each canton

In [90]:
pd.set_option("display.float_format", "{:,.1f}".format)

In [91]:
for canton in cantons_list:
    canton['properties']['factor'] = cantons_factor[canton['properties']['kanton']]

In [92]:
for canton in cantons_list:
    print (canton['properties'])

{'kanton': 'JU', 'kanton_long': 'Jura', 'factor': '4.5'}
{'kanton': 'NE', 'kanton_long': 'Neuenburg', 'factor': '3.0'}
{'kanton': 'BE', 'kanton_long': 'Bern', 'factor': '4.5'}
{'kanton': 'GR', 'kanton_long': 'Graubünden', 'factor': '2.4'}
{'kanton': 'VD', 'kanton_long': 'Waadt', 'factor': '4.7'}
{'kanton': 'BS', 'kanton_long': 'Basel-Stadt', 'factor': '4.0'}
{'kanton': 'ZG', 'kanton_long': 'Zug', 'factor': '7.7'}
{'kanton': 'NW', 'kanton_long': 'Nidwalden', 'factor': '15.0'}
{'kanton': 'AI', 'kanton_long': 'Appenzell Innerrhoden', 'factor': '1.0'}
{'kanton': 'LU', 'kanton_long': 'Luzern', 'factor': '3.5'}
{'kanton': 'SZ', 'kanton_long': 'Schwyz', 'factor': '2.5'}
{'kanton': 'UR', 'kanton_long': 'Uri', 'factor': '7.0'}
{'kanton': 'OW', 'kanton_long': 'Obwalden', 'factor': '12.0'}
{'kanton': 'GL', 'kanton_long': 'Glarus', 'factor': '0.0'}
{'kanton': 'BL', 'kanton_long': 'Basel-Landschaft', 'factor': '11.1'}
{'kanton': 'SO', 'kanton_long': 'Solothurn', 'factor': '6.5'}
{'kanton': 'SH', 'k

* now we attach the dictionary on each councillor to the corresponding canton

In [93]:
pd.set_option("display.float_format", "{:,.0f}".format)

In [95]:
for canton in cantons_list:
    canton['properties']['councillors'] = c_dicts[c_dicts['canton'] == canton['properties']['kanton']].sort_values('num_questions', ascending=False).to_dict(orient="records")

In [96]:
for canton in cantons_list:
    print (canton['properties']['councillors'])

[{'councillor_id': '4074', 'fullname': 'Pierre-Alain Fridez', 'party': 'SP', 'canton': 'JU', 'num_questions': 7.0}, {'councillor_id': '4087', 'fullname': 'Jean-Paul Gschwind', 'party': 'CVP', 'canton': 'JU', 'num_questions': 2.0}]
[{'councillor_id': '4018', 'fullname': 'Jacques-André Maire', 'party': 'SP', 'canton': 'NE', 'num_questions': 4.0}, {'councillor_id': '4187', 'fullname': 'Philippe Bauer', 'party': 'FDP-Liberale', 'canton': 'NE', 'num_questions': 3.0}, {'councillor_id': '4135', 'fullname': 'Raymond Clottu', 'party': 'SVP', 'canton': 'NE', 'num_questions': 3.0}, {'councillor_id': '4201', 'fullname': 'Denis de la Reussille', 'party': 'PdA', 'canton': 'NE', 'num_questions': 2.0}]
[{'councillor_id': '4157', 'fullname': 'Manfred Bühler', 'party': 'SVP', 'canton': 'BE', 'num_questions': 17.0}, {'councillor_id': '4095', 'fullname': 'Regula Rytz', 'party': 'GPS', 'canton': 'BE', 'num_questions': 15.0}, {'councillor_id': '3880', 'fullname': 'Andrea Martina Geissbühler', 'party': 'SVP'

In [ ]:
cantons_json

# 3b Create another json which stores questions

* We are putting this in a separate json / js file in order not to overload the other one

In [97]:
#Some simplification, like above
df['fullname'] = df['firstname'] + " " + df['lastname']

In [98]:
df.head(1)

,affair_id,affair_type,council,legislative_period,language,affair_title,affair_text,answer,councillor_id,firstname,lastname,canton,party,fullname
0,20155540,14,NR,50,de,Prinzip der Wahrung der Interessen der Schweiz...,<p>Die Wahrung der Interessen der Schweiz in d...,,4025,Roland Rino,Büchel,SG,SVP,Roland Rino Büchel


In [102]:
q_dicts = df[['canton', 'fullname', 'party', 'affair_title', 'affair_text']]

In [115]:
q_dicts.head(1)

,canton,fullname,party,affair_title,affair_text
0,SG,Roland Rino Büchel,SVP,Prinzip der Wahrung der Interessen der Schweiz...,<p>Die Wahrung der Interessen der Schweiz in d...


In [111]:
questions_json = {}

In [112]:
for cant_id in cant_ids:
    q_dict = q_dicts[q_dicts['canton'] == cant_id].sort_values('affair_title').to_dict(orient="records")
    questions_json[cant_id] = q_dict

# 4 Save the modified json into a new file

*first, we dump the cantons&councillors json into a new file called `schweizer_kantone.js`

In [116]:
with open('schweizer_kantone.js', 'w') as fp:
    fp.write('var cantonsData = ')
    json.dump(cantons_json, fp)

*second, we dump the questions json into a new file called questions.js

In [117]:
with open('questions.js', 'w') as fp:
    fp.write('var questionsData = ')
    json.dump(questions_json, fp)